<a href="https://colab.research.google.com/github/t0brig01/CSE696FinalProject/blob/main/MoralNeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import pandas as pd
from pandas import concat
from sklearn import preprocessing
print ("TensorFlow version: " + tf.__version__)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE as smt
from sklearn.model_selection import train_test_split



TensorFlow version: 1.15.2


# **Data Loading/Manipulation**

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/t0brig01/CSE696FinalProject/main/TrolleyData.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230475 entries, 0 to 230474
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   _id             230475 non-null  object
 1   Scenario        230475 non-null  object
 2   UserID          230475 non-null  object
 3   Outcome         230475 non-null  int64 
 4   Session_id      230475 non-null  object
 5   Scenario_order  230475 non-null  int64 
 6   Template        230475 non-null  object
 7   answerLeft      230475 non-null  bool  
 8   lang            230475 non-null  object
 9   seenOther       230475 non-null  bool  
 10  country_code    229259 non-null  object
 11  country_full    229259 non-null  object
dtypes: bool(2), int64(2), object(8)
memory usage: 18.0+ MB


In [5]:
df.head()

,_id,Scenario,UserID,Outcome,Session_id,Scenario_order,Template,answerLeft,lang,seenOther,country_code,country_full
0,C3RGCjGPftjMKYHy2,Loop,0000dc12_9518522259818270,1,367585191-9.51852225982e+15,1,Desktop,False,de,True,DE,Germany
1,3F9SfDyofskDPna8R,Footbridge,0000dc12_9518522259818270,0,367585191-9.51852225982e+15,2,Desktop,False,de,True,DE,Germany
2,oz9anPbE6e52TKuJy,Switch,0000dc12_9518522259818270,1,367585191-9.51852225982e+15,3,Desktop,False,de,True,DE,Germany
3,dkAhBaLhzi62Pp6Hp,Footbridge,0002ae2d_2286850331484848,0,1654499857-2.28685033148e+15,1,Desktop,False,en,True,GB,United Kingdom
4,2ussgtw7vCZMNjxSJ,Switch,0002ae2d_2286850331484848,1,1654499857-2.28685033148e+15,2,Desktop,False,en,True,GB,United Kingdom


In [7]:
filtered = df.drop(['Session_id','Template','country_full','_id','answerLeft'], axis=1)
filtered.dropna()
filtered.replace('Loop',0,inplace=True)   #0 is loop trolley 
filtered.replace('Footbridge',1,inplace=True) #1 is footbridge trolley
filtered.replace('Switch',2,inplace=True) #2 is standard trolley

lang = dict()
country = dict()
iLang = 0
iCountry = 0
for index, x in filtered.iterrows():
  if x["lang"] not in lang.keys():
    lang[x["lang"]] = iLang
    iLang = iLang + 1
  if x["country_code"] not in country.keys():
    country[x["country_code"]] = iCountry
    iCountry = iCountry + 1
for key, value in country.items():
  filtered.replace(key,int(value),inplace=True)
for key, value in lang.items():
  filtered.replace(key,int(value),inplace=True)

In [8]:
filtered['country_code'] = filtered['country_code'].astype(int)

In [9]:
filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230475 entries, 0 to 230474
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Scenario        230475 non-null  int64 
 1   UserID          230475 non-null  object
 2   Outcome         230475 non-null  int64 
 3   Scenario_order  230475 non-null  int64 
 4   lang            230475 non-null  int64 
 5   seenOther       230475 non-null  bool  
 6   country_code    230475 non-null  int64 
dtypes: bool(1), int64(5), object(1)
memory usage: 10.8+ MB


In [10]:
filtered.describe()

,Scenario,Outcome,Scenario_order,lang,country_code
count,230475.000000,230475.000000,230475.000000,230475.000000,230475.000000
mean,0.997896,0.700453,1.955301,1.903946,15.234290
std,0.816945,0.458060,0.817113,1.716888,22.201642
min,0.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,1.000000,3.000000
50%,1.000000,1.000000,2.000000,1.000000,6.000000
75%,2.000000,1.000000,3.000000,3.000000,21.000000
max,2.000000,1.000000,3.000000,9.000000,169.000000


In [33]:
def create_dataset(X):
  Xs = X.drop(["UserID","Outcome"],axis = 1)
  Ys = X.drop(["Scenario","UserID","Scenario_order","lang","seenOther","country_code"],axis=1)
  Xs,Ys = Xs.astype(int),Ys.astype(int)
  Xs['seenOther'] = Xs['seenOther'].astype(bool)
  Ys['Outcome'] = Ys['Outcome'].astype(bool)
  return Xs,Ys

In [34]:
interval = int(len(filtered.index)/3)
filtered = filtered.sample(frac=1).reset_index(drop=True)

x_train, y_train = create_dataset(filtered.head(interval))
x_test, y_test = create_dataset(filtered.tail(interval))
x_val, y_val = create_dataset(filtered.loc[interval:interval*2])

In [35]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76825 entries, 0 to 76824
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Scenario        76825 non-null  int64
 1   Scenario_order  76825 non-null  int64
 2   lang            76825 non-null  int64
 3   seenOther       76825 non-null  bool 
 4   country_code    76825 non-null  int64
dtypes: bool(1), int64(4)
memory usage: 2.4 MB


In [36]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76825 entries, 0 to 76824
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Outcome  76825 non-null  bool 
dtypes: bool(1)
memory usage: 75.1 KB


# **Neural Network**

In [59]:
from keras.models import Sequential
from keras import layers
from keras.layers import LSTM, Dense,Conv1D,MaxPooling1D,Flatten,Activation,BatchNormalization,Dropout
from keras.optimizers import RMSprop,SGD,Adam,Nadam
from tensorflow import keras
from keras import optimizers
import tensorflow as tf


In [72]:
def create_model():
  model = Sequential()
  model.add(Dense(50,input_dim=5, activation='relu',kernel_initializer='random_uniform'))
  model.add(BatchNormalization())
  model.add(Dense(100, activation='relu',kernel_initializer='random_uniform'))
  model.add(BatchNormalization())
  model.add(Dense(50, activation='relu',kernel_initializer='random_uniform'))
  model.add(BatchNormalization())
  model.add(Dense(1, activation='sigmoid',kernel_initializer='random_uniform'))
  model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  return model

def create_model2():
  model = Sequential()

  model.add(Dense(24, input_dim=5,kernel_initializer='random_uniform'))

  model.add(Dense(64,activation="relu",kernel_initializer='random_uniform'))
  model.add(BatchNormalization())
  model.add(Dropout(.5))
  model.add(Dense(64, activation="relu",kernel_initializer='random_uniform'))
  model.add(BatchNormalization())

  model.add(Dense(1,activation="sigmoid",kernel_initializer='random_uniform'))
  model.compile(optimizer='rmsprop',loss = 'binary_crossentropy',metrics = ['accuracy'])
  return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=100, batch_size=1000, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True)
results = cross_val_score(pipeline, x_train, y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
57618/57618 [==============================] - 2s 26us/step - loss: 0.6529 - accuracy: 0.6416
Epoch 2/100
57618/57618 [==============================] - 1s 12us/step - loss: 0.6118 - accuracy: 0.6754
Epoch 3/100
57618/57618 [==============================] - 1s 12us/step - loss: 0.5828 - accuracy: 0.6991
Epoch 4/100
57618/57618 [==============================] - 1s 12us/step - loss: 0.5764 - accuracy: 0.7033
Epoch 5/100
57618/57618 [==============================] - 1s 11us/step - loss: 0.5755 - accuracy: 0.7057
Epoch 6/100
57618/57618 [==============================] - 1s 12us/step - loss: 0.5747 - accuracy: 0.7076
Epoch 7/100
57618/57618 [==============================] - 1s 12us/step - loss: 0.5740 - accuracy: 0.7074
Epoch 8/100
57618/57618 [==============================] - 1s 12us/step - loss: 0.5741 - accuracy: 0.7068
Epoch 9/100
57618/57618 [==============================] - 1s 12us/step - loss: 0.5734 - accuracy: 0.7085
Epoch 10/100
57618/57618 [====================

# Other less accurate NNs

In [40]:
#Basic
model = Sequential()
model.add(Dense(60, input_dim=5, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train,y_train,epochs=10, validation_data=(x_val,y_val),batch_size=50)
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print("Results: %.2f%% (%.2f%%)" % (acc*100, loss*100))

Train on 76825 samples, validate on 76826 samples
Epoch 1/10
76825/76825 [==============================] - 4s 53us/step - loss: 0.5968 - accuracy: 0.7019 - val_loss: 0.5829 - val_accuracy: 0.7000
Epoch 2/10
76825/76825 [==============================] - 4s 49us/step - loss: 0.5811 - accuracy: 0.7020 - val_loss: 0.5796 - val_accuracy: 0.6999
Epoch 3/10
76825/76825 [==============================] - 4s 48us/step - loss: 0.5792 - accuracy: 0.7017 - val_loss: 0.5806 - val_accuracy: 0.6999
Epoch 4/10
76825/76825 [==============================] - 4s 48us/step - loss: 0.5781 - accuracy: 0.7033 - val_loss: 0.5782 - val_accuracy: 0.7002
Epoch 5/10
76825/76825 [==============================] - 4s 49us/step - loss: 0.5781 - accuracy: 0.7029 - val_loss: 0.5788 - val_accuracy: 0.7003
Epoch 6/10
76825/76825 [==============================] - 4s 49us/step - loss: 0.5776 - accuracy: 0.7027 - val_loss: 0.5791 - val_accuracy: 0.7022
Epoch 7/10
76825/76825 [==============================] - 4s 49us/st

In [43]:
model = Sequential()

model.add(Dense(24, input_dim=5,kernel_initializer='random_uniform'))

model.add(layers.Dense(64,activation="relu",kernel_initializer='random_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(.5))
model.add(layers.Dense(64, activation="relu",kernel_initializer='random_uniform'))
model.add(layers.BatchNormalization())

model.add(Dense(1,activation="sigmoid",kernel_initializer='random_uniform'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(x_train, y_train, epochs=10)
# evaluate the keras model
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f%%' % (accuracy*100))

Epoch 1/10
76825/76825 [==============================] - 5s 64us/step - loss: 0.5951 - accuracy: 0.6942
Epoch 2/10
76825/76825 [==============================] - 5s 60us/step - loss: 0.5838 - accuracy: 0.7003
Epoch 3/10
76825/76825 [==============================] - 5s 60us/step - loss: 0.5828 - accuracy: 0.7012
Epoch 4/10
76825/76825 [==============================] - 5s 61us/step - loss: 0.5817 - accuracy: 0.7016
Epoch 5/10
76825/76825 [==============================] - 5s 60us/step - loss: 0.5816 - accuracy: 0.7006
Epoch 6/10
76825/76825 [==============================] - 5s 60us/step - loss: 0.5811 - accuracy: 0.7011
Epoch 7/10
76825/76825 [==============================] - 5s 60us/step - loss: 0.5813 - accuracy: 0.7003
Epoch 8/10
76825/76825 [==============================] - 5s 61us/step - loss: 0.5808 - accuracy: 0.7006
Epoch 9/10
76825/76825 [==============================] - 5s 61us/step - loss: 0.5813 - accuracy: 0.6998
Epoch 10/10
76825/76825 [==============================

# Epochs vs Batch Experiment

In [ ]:
def buildModel():
  model = Sequential()
  model.add(Dense(60, input_dim=5, activation='relu'))
  model.add(Dense(40, activation='relu'))
  model.add(Dense(10, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
acc = dict()
for batch in range(1,15): #batch size 
  for epoch in range(1,15): #epochs
    print("Epochs: {0} \t Batch Size: {1}".format(10*epoch,16*batch))
    model = buildModel()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=(10*epoch), batch_size=(16*batch),verbose=0)
    loss, accuracy = model.evaluate(x_test, y_test,verbose=0)
    key = "epochs{0}batch{1}".format(10*epoch,16*batch)
    acc[key] = accuracy*100
    print('Accuracy: %.2f%%' % (accuracy*100))

Epochs: 10 	 Batch Size: 16
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Accuracy: 70.39%
Epochs: 20 	 Batch Size: 16
Accuracy: 70.27%
Epochs: 30 	 Batch Size: 16
Accuracy: 70.59%
Epochs: 40 	 Batch Size: 16
Accuracy: 70.60%
Epochs: 50 	 Batch Size: 16
Accuracy: 70.44%
Epochs: 60 	 Batch Size: 16
Accuracy: 70.70%
Epochs: 70 	 Batch Size: 16
Accuracy: 70.38%
Epochs: 80 	 Batch Size: 16
Accuracy: 70.79%
Epochs: 90 	 Batch Size: 16
Accuracy: 70.85%
Epochs: 100 	 Batch Size: 16
Accuracy: 70.80%
Epochs: 110 	 Batch Size: 16
Accuracy: 70.66%
Epochs: 120 	 Batch Size: 16
Accuracy: 70.85%
Epochs: 130 	 Batch Size: 16
Accuracy: 70.72%
Epochs: 140 	 Batch Size: 16
Accuracy: 70.70%
Epochs: 10 	 Batch Size: 32
Accuracy: 70.41%
Epochs: 20 	 Batch Size: 32
Accuracy: 70.28%
Epochs: 30 	 Batch Size: 32
Accuracy: 70.56%
Epochs: 40 	 Batch Size: 32
Accuracy: 70.51%
Epochs: 5

# Hyperparameters


In [ ]:
!pip install git+https://github.com/keras-team/keras-tuner
# !pip uninstall tensorflow -y
!pip install tensorflow-gpu
!pip install gdown

  Cloning https://github.com/keras-team/keras-tuner to /tmp/pip-req-build-dbblphoy
  Running command git clone -q https://github.com/keras-team/keras-tuner /tmp/pip-req-build-dbblphoy
  Created wheel for keras-tuner: filename=keras_tuner-1.0.3-cp37-none-any.whl size=93604 sha256=91d27f0aa114968deee6dce7e40b81365f98a8d465db62472f57a3bc743cbab0
  Stored in directory: /tmp/pip-ephem-wheel-cache-s55ow8p_/wheels/6c/55/2d/6e178386cb7a2d7da5a7059752a2b58791705c9c8718c5f07a
Successfully built keras-tuner
     |████████████████████████████████| 394.3MB 31kB/s 


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pprint import pprint
from kerastuner.tuners import RandomSearch, BayesianOptimization

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 16, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
def tune_optimizer_model(hp):
    model = keras.Sequential()

    model.add(keras.layers.Dense(24, input_dim=5,kernel_initializer='random_uniform'))

    model.add(keras.layers.Dense(64,activation="relu",kernel_initializer='random_uniform'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    model.add(keras.layers.Dense(64, activation="relu",kernel_initializer='random_uniform'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Dense(1,activation="sigmoid",kernel_initializer='random_uniform'))
    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    model.compile(
        optimizer=optimizer,
        loss = 'binary_crossentropy',
        metrics = ['accuracy'])
    return model

In [ ]:
from kerastuner.tuners import RandomSearch
MAX_TRIALS = 20
EXECUTIONS_PER_TRIAL = 5

tuner = RandomSearch(
    tune_optimizer_model,
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTIONS_PER_TRIAL,
    directory='test_dir',
    project_name='tune_optimizer',
    seed=RANDOM_SEED
)
tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project test_dir/tune_optimizer/oracle.json
Search space summary
Default search space size: 1
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd', 'rmsprop'], 'ordered': False}


In [ ]:
TRAIN_EPOCHS = 20

tuner.search(x=x_train,
             y=y_train,
             epochs=TRAIN_EPOCHS,
             validation_data=(x_test, y_test))

Trial 3 Complete [00h 09m 51s]
val_accuracy: 0.7029742956161499

Best val_accuracy So Far: 0.7031591296195984
Total elapsed time: 00h 31m 44s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in test_dir/tune_optimizer
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7031591296195984
Trial summary
Hyperparameters:
optimizer: adam
Score: 0.7030419826507568
Trial summary
Hyperparameters:
optimizer: sgd
Score: 0.7029742956161499


In [ ]:
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

{'optimizer': 'rmsprop'}

In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                144       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1600      
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6